In [146]:
pip install matplotlib

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [147]:
pip install statsmodels

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [148]:
pip install tqdm

You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import statsmodels.api as sm
from tqdm import tqdm
from multiprocessing.dummy import Pool
import statsmodels.formula.api as smf
pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

In [2]:
crsp_df = pd.read_csv("crsp.zip", compression='zip',header=0,
             parse_dates = ['date','DCLRDT','RCRDDT'],
             dtype={"PRC": np.float64,
                  "VOL": np.float64,
                  "SHROUT": np.float64,
                  "DIVAMT": np.float64,
                  "SPREAD": np.float64,
                  "SHRCD":str,
                  "DISTCD":str,
                  "COMNAM": str,
                  "TICKER": str,
                   'CUSIP': str})
#Data Preprocessing
crsp_df = crsp_df[(crsp_df.SHRCD.isin(('10','11')))]
crsp_df = crsp_df[(crsp_df.date >= '1927-01-01')]
crsp_df = crsp_df[(crsp_df.date <= '2011-12-31')]
cusips_DISTCD_12 = crsp_df[crsp_df.DISTCD.apply(lambda x: x[:2]=='12' if isinstance(x, str) else False)].CUSIP.unique()
crsp_df = crsp_df[crsp_df.CUSIP.isin(cusips_DISTCD_12)]

crsp_df.PRC = crsp_df.PRC.abs()


crsp_df['year'] = crsp_df['date'].dt.year
crsp_df['month'] = crsp_df['date'].dt.month
crsp_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_df['PRC_t-1'] = crsp_df.groupby('CUSIP')['PRC'].shift(1)
crsp_df.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,CUSIP,DCLRDT,RCRDDT,DISTCD,DIVAMT,PRC,VOL,RET,SHROUT,SPREAD,RETX,year,month,PRC_t-1
1359100,25881,1970-11-30,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,318.0,C,2655.0,NaN,C,1970,11,NaN
1359101,25881,1970-12-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,10.000,646.0,0.111111,2655.0,NaN,0.111111,1970,12,9.000
1359102,25881,1971-01-29,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,636.0,-0.100000,2655.0,NaN,-0.100000,1971,1,10.000
1359103,25881,1971-02-26,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,11.125,2155.0,0.236111,2655.0,NaN,0.236111,1971,2,9.000
1359104,25881,1971-03-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.750,2062.0,-0.123596,2655.0,NaN,-0.123596,1971,3,11.125


In [3]:
crsp_div_df = crsp_df[['date','CUSIP','DCLRDT','RCRDDT','DISTCD','DIVAMT','PRC','PRC_t-1','RET','SHROUT','month']].copy()
crsp_div_df 
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df = crsp_div_df.groupby(by=['CUSIP','date']).agg({
     'DCLRDT': 'last',
     'RCRDDT': 'last',
     'DISTCD': 'last',
     'DIVAMT': 'sum',
     'RET':'max',
     'PRC': 'last',
     'PRC_t-1': 'last',
     'SHROUT': 'last'})
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df['freq'] = None
crsp_div_df['freq'] = crsp_div_df[crsp_div_df.RCRDDT.isna()==False]['DISTCD'].apply(lambda x: x[2:3] if x is not None else None)
crsp_div_df['freq'] = crsp_div_df.groupby(by=['CUSIP']).fillna(method='ffill', limit=11)['freq']

fil_C = (crsp_div_df["PRC_t-1"] >= 5) & (crsp_div_df.freq < '6')
crsp_div_df = crsp_div_df[fil_C]

crsp_div_df['MCAP'] = crsp_div_df['PRC'] * crsp_div_df['SHROUT']
crsp_div_df

DCLRDT     RCRDDT DISTCD  DIVAMT        RET     PRC  \
CUSIP    date                                                                 
00003210 1977-06-30 1977-05-25 1977-06-24   1232    0.05   0.069388   6.500   
         1977-07-29        NaT        NaT   None    0.00   0.230769   8.000   
         1977-08-31        NaT        NaT   None    0.00   0.046875   8.375   
         1977-09-30 1977-08-02 1977-09-23   1232    0.05   0.080597   9.000   
         1977-10-31        NaT        NaT   None    0.00  -0.041667   8.625   
...                        ...        ...    ...     ...        ...     ...   
N5946510 2008-01-31        NaT        NaT   None    0.00   0.055477  14.840   
         2008-02-29        NaT        NaT   None    0.00  -0.202156  11.840   
         2008-03-31        NaT        NaT   None    0.00  -0.020270  11.600   
         2008-04-30        NaT        NaT   None    0.00   0.135345  13.170   
         2008-05-30        NaT        NaT   None    0.00   0.013667  13.350   

                     PRC_t-1    SHROUT freq         MCAP  
CUSIP    date                                             
00003210 1977-06-30    6.125    2209.0    3    14358.500  
         1977-07-29    6.500    2209.0    3    17672.000  
         1977-08-31    8.000    2209.0    3    18500.375  
         1977-09-30    8.375    2223.0    3    20007.000  
         1977-10-31    9.000    2223.0    3    19173.375  
...                      ...       ...  ...          ...  
N5946510 2008-01-31   14.060  304332.0    3  4516286.880  
         2008-02-29   14.840  304332.0    3  3603290.880  
         2008-03-31   11.840  304439.0    3  3531492.400  
         2008-04-30   11.600  304439.0    3  4009461.630  
         2008-05-30   13.170  304449.0    3  4064394.150  

[1494129 rows x 10 columns]

In [18]:
# a = pd.Series(['1.0',2.0,100.0,'C','B'])
# a < '65'
'B' < '65'

False

In [19]:
crsp_div_df['div_yr']=np.where(crsp_div_df['freq']=='6',True,False )
crsp_div_df['div_semi_annual']=np.where(crsp_div_df['freq']=='4',True,False)
crsp_div_df['div_quarterly']=np.where(crsp_div_df['freq']=='3',True,False)
crsp_div_df['div_unknown']=np.where(crsp_div_df['freq']=='0',True,False )
crsp_div_df['div_monthly']=np.where(crsp_div_df['freq']=='2',True,False )
crsp_temp=crsp_div_df


In [20]:
crsp_temp['lag_u_1']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(1)
crsp_temp['lag_u_2']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(2)
crsp_temp['lag_u_3']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(3)
crsp_temp['lag_u_4']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(4)
crsp_temp['lag_u_5']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(5)
crsp_temp['lag_u_6']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(6)
crsp_temp['lag_u_7']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(7)
crsp_temp['lag_u_8']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(8)
crsp_temp['lag_u_9']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(9)
crsp_temp['lag_u_10']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(10)
crsp_temp['lag_u_11']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(11)
crsp_temp['lag_u_12']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(12)

crsp_temp['lag_quarterly']=crsp_temp.groupby('CUSIP')['div_quarterly'].shift(3)
crsp_temp['lag_semi_annual_six']=crsp_temp.groupby('CUSIP')['div_semi_annual'].shift(6)
crsp_temp['lag_semi_annual_twl']=crsp_temp.groupby('CUSIP')['div_semi_annual'].shift(12)
crsp_temp['lag_div_yr']=crsp_temp.groupby('CUSIP')['div_yr'].shift(12)
crsp_temp['lag_1']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(1)
crsp_temp['lag_4']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(4)
crsp_temp['lag_7']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(7)
crsp_temp['lag_10']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(10)

crsp_div_df['lag_month']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(1)

crsp_temp

DCLRDT     RCRDDT DISTCD  DIVAMT        RET     PRC  \
CUSIP    date                                                                 
00003210 1977-06-30 1977-05-25 1977-06-24   1232    0.05   0.069388   6.500   
         1977-07-29        NaT        NaT   None    0.00   0.230769   8.000   
         1977-08-31        NaT        NaT   None    0.00   0.046875   8.375   
         1977-09-30 1977-08-02 1977-09-23   1232    0.05   0.080597   9.000   
         1977-10-31        NaT        NaT   None    0.00  -0.041667   8.625   
...                        ...        ...    ...     ...        ...     ...   
N5946510 2008-01-31        NaT        NaT   None    0.00   0.055477  14.840   
         2008-02-29        NaT        NaT   None    0.00  -0.202156  11.840   
         2008-03-31        NaT        NaT   None    0.00  -0.020270  11.600   
         2008-04-30        NaT        NaT   None    0.00   0.135345  13.170   
         2008-05-30        NaT        NaT   None    0.00   0.013667  13.350   

                     PRC_t-1    SHROUT freq         MCAP  ...  lag_u_12  \
CUSIP    date                                             ...             
00003210 1977-06-30    6.125    2209.0    3    14358.500  ...       NaN   
         1977-07-29    6.500    2209.0    3    17672.000  ...       NaN   
         1977-08-31    8.000    2209.0    3    18500.375  ...       NaN   
         1977-09-30    8.375    2223.0    3    20007.000  ...       NaN   
         1977-10-31    9.000    2223.0    3    19173.375  ...       NaN   
...                      ...       ...  ...          ...  ...       ...   
N5946510 2008-01-31   14.060  304332.0    3  4516286.880  ...     False   
         2008-02-29   14.840  304332.0    3  3603290.880  ...     False   
         2008-03-31   11.840  304439.0    3  3531492.400  ...     False   
         2008-04-30   11.600  304439.0    3  4009461.630  ...     False   
         2008-05-30   13.170  304449.0    3  4064394.150  ...     False   

                     lag_quarterly  lag_semi_annual_six  lag_semi_annual_twl  \
CUSIP    date                                                                  
00003210 1977-06-30            NaN                  NaN                  NaN   
         1977-07-29            NaN                  NaN                  NaN   
         1977-08-31            NaN                  NaN                  NaN   
         1977-09-30           True                  NaN                  NaN   
         1977-10-31           True                  NaN                  NaN   
...                            ...                  ...                  ...   
N5946510 2008-01-31           True                False                False   
         2008-02-29           True                False                False   
         2008-03-31           True                False                False   
         2008-04-30           True                False                False   
         2008-05-30           True                False                False   

                     lag_div_yr  lag_1  lag_4  lag_7 lag_10 lag_month  
CUSIP    date                                                          
00003210 1977-06-30         NaN    NaN    NaN    NaN    NaN       NaN  
         1977-07-29         NaN  False    NaN    NaN    NaN     False  
         1977-08-31         NaN  False    NaN    NaN    NaN     False  
         1977-09-30         NaN  False    NaN    NaN    NaN     False  
         1977-10-31         NaN  False  False    NaN    NaN     False  
...                         ...    ...    ...    ...    ...       ...  
N5946510 2008-01-31       False  False  False  False  False     False  
         2008-02-29       False  False  False  False  False     False  
         2008-03-31       False  False  False  False  False     False  
         2008-04-30       False  False  False  False  False     False  
         2008-05-30       False  False  False  False  False     False  

[1494129 rows x 36 columns]

In [21]:
# Strategies 1: Long predicted dividend and short all other companies
crsp_temp['long_stocks'] = (crsp_temp['lag_u_3'] ==True) | (crsp_temp['lag_u_6'] ==True) | ( crsp_temp['lag_u_9'] ==True ) | (crsp_temp['lag_u_12'] ==True ) | ( crsp_temp['lag_quarterly'] ==True ) | (crsp_temp['lag_semi_annual_six'] ==True ) | (crsp_temp['lag_semi_annual_twl'] ==True) |(crsp_temp['lag_div_yr'] ==True)

# crsp_temp['long_stocks'] = (crsp_temp['div_yr'] ==True) | (crsp_temp['div_semi_annual'] ==True) | ( crsp_temp['div_quarterly'] ==True ) | (crsp_temp['div_unknown'] ==True ) 

crsp_temp['short_stocks'] = np.where(crsp_temp['long_stocks']==False,True,False)

# Strategies 2: Long predicted dividend and short all other dividend payers companies
# crsp_temp['short_stocks'] = (crsp_temp['lag_u_1'] ==True) | (crsp_temp['lag_u_2'] ==True) | ( crsp_temp['lag_u_4'] ==True ) | (crsp_temp['lag_u_5'] ==True ) | ( crsp_temp['lag_u_8'] ==True ) | (crsp_temp['lag_u_10'] ==True ) | (crsp_temp['lag_u_11'] ==True )| (crsp_div_df['lag_month']==True)

# Strategies 3: Long predicted dividend and short all months occurring 1,4,7 or 10 months ago
# crsp_temp['short_stocks'] = (crsp_temp['lag_1'] ==True) | (crsp_temp['lag_4'] ==True) | ( crsp_temp['lag_7'] ==True ) | (crsp_temp['lag_10'] ==True ) 
# crsp_temp['short_stocks'].describe()



In [37]:
def long_weights(group):
    # Select all stocks with long position from the group
    longs = group.loc[group['long_stocks'] == True]
    # Get market capitalization of each long stock
    mktcap = longs['SHROUT'] * longs['PRC']
    # Calculate long weights by market captalization
    longweights = mktcap / mktcap.sum()
    # Assign weights for the group of long stocks
    group.loc[group['long_stocks']==True, 'long_weight'] = longweights
    group['long_weight'] = group['long_weight'].shift(1)
    return group

def short_weights(group):
    # Select all stocks with short position from the group
    shorts = group.loc[group['short_stocks'] == True]
    # Get market capitalization of each long stock
    mktcap = shorts['SHROUT'] * shorts['PRC']
    # Calculate long weights by market captalization
    shortweights = mktcap / mktcap.sum()
    # Assign weights for the group of long stocks
    group.loc[group['short_stocks']==True, 'short_weight'] = shortweights
    return group

def cal_return_vw(group):
    # group=group[(group.RET<='65')]
    group =  group[( group['RET'].apply(lambda x: str(x)[-1].isdigit()) )]
    group['port_return_long_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['long_weight'])
    longport = pd.pivot_table(group, values='port_return_long_vw', index='date', aggfunc=np.sum)

    group['port_return_short_vw'] = pd.to_numeric(group['RET']) * pd.to_numeric(group['short_weight'])
    shortport = pd.pivot_table(group, values='port_return_short_vw', index='date', aggfunc=np.sum)

    data=pd.merge(longport, shortport, on='date')
    # print(data)
    data['longshort_eq'] = data['port_return_long_vw'] - data['port_return_short_vw']
    return longport,shortport,data

def cal_return_eq(group):
    #Kobe Updated part, 
    longs = group.loc[group['long_stocks'] == True]
    longs =  longs[( longs['RET'].apply(lambda x: str(x)[-1].isdigit()) )] # this function for filter out number
    longs['RET'] = longs['RET'].astype('float64')
    longport = longs.RET.groupby('date').mean() 


    shorts = group.loc[group['short_stocks'] == True]
    shorts =  shorts[( shorts['RET'].apply(lambda x: str(x)[-1].isdigit()) )]
    shorts['RET'] = shorts['RET'].astype('float64')
    shortport = shorts.RET.groupby('date').mean()
    
    data=pd.merge(longport, shortport, on='date', suffixes=['_long', '_short'])
    data['longshort_eq'] = data.RET_long  - data.RET_short
    print(data)
    # print(shortport)
    # print(longshortport)
    
    return longport,shortport,data

panel=long_weights(crsp_temp)
panel=short_weights(panel)
longport_vw,shortport_vw,longshortport_vw=cal_return_vw(panel)
longport_eq,shortport_eq,longshortport_eq=cal_return_eq(crsp_temp)



            RET_long  RET_short  longshort_eq
date                                         
1927-04-30  0.018177  -0.000182      0.018359
1927-05-31  0.051929   0.050309      0.001620
1927-06-30 -0.020331  -0.003609     -0.016722
1927-07-30  0.058961   0.056138      0.002823
1927-08-31  0.011821  -0.002201      0.014022
...              ...        ...           ...
2011-08-31 -0.060211  -0.089215      0.029004
2011-09-30 -0.085287  -0.104952      0.019665
2011-10-31  0.125390   0.136289     -0.010899
2011-11-30  0.003480   0.015668     -0.012188
2011-12-30  0.007794   0.001967      0.005826

[1017 rows x 3 columns]


In [23]:
ff4=pd.read_csv('four_factor_monthly.csv',index_col='date')
ff4.index = pd.to_datetime(ff4.index)

In [39]:
pd.merge(ff4, longport_eq, on='date')

,mkt_rf,smb,hml,rf,Mom,RET
date,,,,,,
1927-04-30,0.005123,0.004924,0.004737,0.002503,0.041533,0.018177
1927-05-31,0.053512,0.013014,0.047367,0.003004,0.035404,0.051929
1927-06-30,-0.023892,0.003808,-0.015245,0.002503,0.020166,-0.020331
1927-08-31,0.020027,-0.007575,-0.035480,0.002704,0.026867,0.011821
1927-09-30,0.048230,-0.033924,-0.006663,0.002002,0.027053,0.035153
...,...,...,...,...,...,...
2011-06-30,-0.017457,-0.000473,-0.004444,0.000000,0.018392,-0.008909
2011-08-31,-0.059974,-0.028293,-0.024706,0.000000,-0.002962,-0.060211
2011-09-30,-0.075968,-0.034320,-0.015275,0.000000,0.023169,-0.085287


In [38]:
# equal_weighted_long
# data=pd.merge(ff4, longport_eq, on='date')

(
    smf.ols('RET ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longport_eq, on='date'))
    .fit().summary()
)

# equal_weighted_short
# (
#     smf.ols('port_return_short_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, shortport_eq, on='date'))
#     .fit().summary()
# )
# equal_weighted_diff
# (
#     smf.ols('longshort_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longshortport_eq, on='date'))
#     .fit().summary()
# )

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RET   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     2463.
Date:                Sat, 29 May 2021   Prob (F-statistic):               0.00
Time:                        23:08:06   Log-Likelihood:                 2064.2
No. Observations:                 744   AIC:                            -4118.
Df Residuals:                     739   BIC:                            -4095.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0050      0.001      8.844      0.000       0.004       0.006
hml            0.1746      0.017     10.257      0.000       0.141       0.208
mkt_rf         0.8684      0.011     75.592      0.000       0.846       0.891
smb            0.2871      0.018     15.826      0.000       0.251       0.323
Mom           -0.0685      0.013     -5.320      0.000      -0.094      -0.043
==============================================================================
Omnibus:                      245.721   Durbin-Watson:                   1.797
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4529.961
Skew:                          -0.994   Prob(JB):                         0.00
Kurtosis:                      14.924   Cond. No.                         33.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
# value_weighted_long
data=pd.merge(ff4, longport_eq, on='date')

(
    smf.ols('port_return_long_vw ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longport_vw, on='date'))
    .fit().summary()
)

# value_weighted_short
# (
#     smf.ols('port_return_short_vw ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, shortport_vw, on='date'))
#     .fit().summary()
# )
# value_weighted_diff
# (
#     smf.ols('longshort_eq ~ 1 + hml + mkt_rf + smb + Mom', data=pd.merge(ff4, longshortport_vw, on='date'))
#     .fit().summary()
# )

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     port_return_long_vw   R-squared:                       0.226
Model:                             OLS   Adj. R-squared:                  0.222
Method:                  Least Squares   F-statistic:                     54.22
Date:                 Sat, 29 May 2021   Prob (F-statistic):           4.12e-40
Time:                         22:48:50   Log-Likelihood:                 6155.6
No. Observations:                  747   AIC:                        -1.230e+04
Df Residuals:                      742   BIC:                        -1.228e+04
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.488e-06   2.41e-06      2.281      0.023    7.66e-07    1.02e-05
hml           -0.0003   7.18e-05     -3.915      0.000      -0.000      -0.000
mkt_rf         0.0006   4.86e-05     13.328      0.000       0.001       0.001
smb           -0.0003   7.67e-05     -4.327      0.000      -0.000      -0.000
Mom           -0.0001   5.44e-05     -2.389      0.017      -0.000   -2.32e-05
==============================================================================
Omnibus:                      274.485   Durbin-Watson:                   1.753
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7650.527
Skew:                          -1.037   Prob(JB):                         0.00
Kurtosis:                      18.540   Cond. No.                         34.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [160]:
# panel B do one strategies 1 & 2 
# CAPM Alpha
(
    smf.ols('port_return_long_eq ~ 1 + mkt_rf', data=pd.merge(ff4, longport_eq, on='date'))
    .fit().summary()
)



<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     port_return_long_eq   R-squared:                       0.000
Model:                             OLS   Adj. R-squared:                 -0.001
Method:                  Least Squares   F-statistic:                   0.04730
Date:                 Sat, 29 May 2021   Prob (F-statistic):              0.828
Time:                         01:24:25   Log-Likelihood:                -2844.0
No. Observations:                  744   AIC:                             5692.
Df Residuals:                      742   BIC:                             5701.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.6335      0.408     38.324      0.000      14.833      16.434
mkt_rf        -1.6002      7.358     -0.217      0.828     -16.045      12.845
==============================================================================
Omnibus:                     4247.691   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.673
Skew:                           0.156   Prob(JB):                     1.10e-13
Kurtosis:                       1.648   Cond. No.                         18.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [161]:
# 3-factor Alpha
(
    smf.ols('port_return_short_eq ~ 1 + hml + mkt_rf + smb', data=pd.merge(ff4, shortport_eq, on='date'))
    .fit().summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     port_return_short_eq   R-squared:                       0.003
Model:                              OLS   Adj. R-squared:                 -0.001
Method:                   Least Squares   F-statistic:                    0.7856
Date:                  Sat, 29 May 2021   Prob (F-statistic):              0.502
Time:                          01:24:25   Log-Likelihood:                -1279.6
No. Observations:                   746   AIC:                             2567.
Df Residuals:                       742   BIC:                             2586.
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4448      0.050     29.048      0.000       1.347       1.542
hml            1.1131      1.482      0.751      0.453      -1.796       4.022
mkt_rf        -0.5628      0.986     -0.571      0.568      -2.499       1.373
smb           -1.6585      1.614     -1.027      0.305      -4.828       1.511
==============================================================================
Omnibus:                      109.431   Durbin-Watson:                   0.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.886
Skew:                           1.102   Prob(JB):                     8.56e-35
Kurtosis:                       3.440   Cond. No.                         33.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""